#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle



/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
#load the datast
#[OLECTRA,"LT","CONCOR","ELGIEQUIP","IOC","BEL","TATAELXSI"]
stock_name = "IOC"
data = yfinance.download (tickers = "{}.NS".format(stock_name),start="2000-03-06",
                               interval = "1d", group_by = 'ticker', auto_adjust = True)

data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2023-06-22,92.000000,92.849998,91.750000,92.000000,9694937
2023-06-23,92.300003,92.400002,88.849998,89.300003,18134295
2023-06-26,89.300003,89.949997,88.699997,89.550003,10963542
2023-06-27,89.750000,90.500000,88.900002,89.349998,10668117
2023-06-30,90.949997,91.949997,90.650002,91.300003,12581625


#Data preparation

In [3]:
#transform the date variable
data["Data"] = pd.to_datetime(data.index, format = "%m/%d/%Y")
data.Data

Date
2000-03-06   2000-03-06
2000-03-07   2000-03-07
2000-03-08   2000-03-08
2000-03-09   2000-03-09
2000-03-10   2000-03-10
                ...    
2023-06-22   2023-06-22
2023-06-23   2023-06-23
2023-06-26   2023-06-26
2023-06-27   2023-06-27
2023-06-30   2023-06-30
Name: Data, Length: 5821, dtype: datetime64[ns]

In [4]:
#renaming variables
data = data.rename(columns = {'Close': 'y',
                     'Data': 'ds'})
data.tail(10)

,Open,High,Low,y,Volume,ds
Date,,,,,,
2023-06-15,93.300003,93.699997,92.599998,92.900002,8742159,2023-06-15
2023-06-16,93.000000,93.199997,91.900002,92.150002,15146569,2023-06-16
2023-06-19,92.150002,92.849998,91.650002,91.900002,6800507,2023-06-19
2023-06-20,91.900002,92.599998,91.300003,91.900002,9394894,2023-06-20
2023-06-21,91.949997,92.849998,91.699997,92.000000,9499893,2023-06-21
2023-06-22,92.000000,92.849998,91.750000,92.000000,9694937,2023-06-22
2023-06-23,92.300003,92.400002,88.849998,89.300003,18134295,2023-06-23
2023-06-26,89.300003,89.949997,88.699997,89.550003,10963542,2023-06-26
2023-06-27,89.750000,90.500000,88.900002,89.349998,10668117,2023-06-27


#Holidays

In [5]:
#Easter
#easter_dates = df[df.Easter == 1].ds
#easter = pd.DataFrame({'holiday': 'easter',
#                      'ds': pd.to_datetime(easter_dates),
#                      'lower_window': -5,
#                      'upper_window': 2})
#easter.head(2)

In [6]:
#Thanksgiving
#thanksgiving_dates = df[df.Thanksgiving == 1].ds
#thanksgiving = pd.DataFrame({'holiday': 'thanksgiving',
 #                      'ds': pd.to_datetime(thanksgiving_dates),
  #                     'lower_window': -5,
   #                    'upper_window': 2})
#thanksgiving.head(2)

In [7]:
#combining holidays
#holidays = pd.concat([easter, thanksgiving])
#holidays

In [8]:
#create training and test set
test_days = 2000
training_set = data.iloc[:-test_days,:]
test_set = data.iloc[-test_days:,:]

In [9]:
 training_set.tail(1)
#test_set.tail(1)

,Open,High,Low,y,Volume,ds
Date,,,,,,
2015-05-26,35.580971,35.652325,34.984642,35.438259,2215770,2015-05-26


#Prophet

In [10]:
m = Prophet(
            #holidays = holidays,
            seasonality_mode = 'multiplicative',
            seasonality_prior_scale = 20,
            changepoint_prior_scale = 0.1,
           # holidays_prior_scale = 10
           )
#m.add_regressor('Christmas')
m.fit(training_set)


18:01:17 - cmdstanpy - INFO - Chain [1] start processing
18:01:25 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
mFullData = Prophet(
            #holidays = holidays,
            seasonality_mode = 'multiplicative',
            seasonality_prior_scale = 20,
            changepoint_prior_scale = 0.1,
           # holidays_prior_scale = 10
           )
#m.add_regressor('Christmas')
mFullData.fit(data)

18:01:25 - cmdstanpy - INFO - Chain [1] start processing
18:01:33 - cmdstanpy - INFO - Chain [1] done processing


In [12]:

# Save the trained model to a file
with open('./TrainedModel/prophet/{}prophet_model_2.pkl'.format(stock_name), 'wb') as f:
    pickle.dump(mFullData, f)

In [13]:
#create future df
future = m.make_future_dataframe(periods = test_days,
                                 freq = "B")

#create future df
future_full = mFullData.make_future_dataframe(periods = 0,
                                 freq = "B")

In [14]:
future_full.tail(1)

,ds
5820,2023-06-30


In [15]:
future = pd.concat([future], axis = 1)
future_full = pd.concat([future_full], axis = 1)

 

In [16]:
future_full

,ds
0,2000-03-06
1,2000-03-07
2,2000-03-08
3,2000-03-09
4,2000-03-10
...,...
5816,2023-06-22
5817,2023-06-23
5818,2023-06-26
5819,2023-06-27


In [ ]:
#forecast
forecast = m.predict(future)
forecast.tail()
#forecast full data
forecast_full = mFullData.predict(future_full)
forecast_full.tail()

In [ ]:
data = data.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
data

In [ ]:
#get some variables
prophet_variables = forecast.loc[:, ["trend", "weekly",  "multiplicative_terms"]]
df_xgb = pd.concat([data, prophet_variables], axis = 1)
df_xgb.tail(1)
prophet_variables_full = forecast_full.loc[:, ["trend", "weekly",  "multiplicative_terms"]]
df_xgb_full = pd.concat([data, prophet_variables_full], axis = 1)
#df_xgb_full.tail(1)
df_xgb_full.tail(1)


#XGBoost preparation

In [ ]:
#create training and test set
test_days = 100
training_set = df_xgb.iloc[:-test_days,:]
test_set = df_xgb.iloc[-test_days:,:]

In [ ]:
#isolate X and y
y_train = training_set.y
y_test = test_set.y 
X_train = training_set.iloc[:, 2:]
X_test = test_set.iloc[:, 2:]
#full data 
y_train_full = df_xgb_full.y
x_train_full = df_xgb_full.iloc[:, 2:]


In [ ]:
x_train_full = x_train_full.drop(columns=["ds"])
X_train = X_train.drop(columns=["ds"])
X_test = X_test.drop(columns=["ds"])
#x_train_full = x_train_full.drop(columns=["ds"])

In [ ]:
#Matrices
import xgboost as xgb
Train = xgb.DMatrix(X_train, label = y_train)
Test = xgb.DMatrix(X_test, label = y_test)

In [ ]:
#Matrices FULL DATA
Train_full = xgb.DMatrix(x_train_full, label = y_train_full)


In [ ]:
#set the parameters
parameters = {'learning_rate': 0.3,
              'max_depth': 6,
              'colsample_bytree': 1,
              'subsample': 1,
              'min_child_weight' : 1,
              'gamma': 0,
              'random_state': 1502,
              'eval_metric': "rmse",
              'objective': "reg:squarederror"}

#XGBoost

In [ ]:
model = xgb.train(params = parameters,
                  dtrain = Train,
                  num_boost_round = 40,
                  evals = [(Test, "y")])

In [ ]:
#FULL
model_full = xgb.train(params = parameters,
                  dtrain = Train_full,
                  num_boost_round = 40,
                  evals = [(Test, "y")])

In [ ]:
Train_full

In [ ]:
#forecasting
predictions = pd.Series(model.predict(Test), name = "XGBoost")
predictions.index  =test_set.ds

In [ ]:

# Save the trained model to a file
with open('./TrainedModel/prophet/{}_prophet_model_1.pkl'.format(stock_name), 'wb') as f:
    pickle.dump(model_full, f)

In [ ]:
#f_predictions = pd.Series(model.predict(Future), name = "XGBoost")
#f_predictions

#VIZ and assessment

In [ ]:
#Visualization
training_set.index = training_set.ds
test_set.index = test_set.ds

In [ ]:
#forecasting
predictions = pd.Series(model.predict(Test), name = "XGBoost")
predictions.index  =test_set.ds

In [ ]:
training_set.y['2020-01-01':].plot(figsize = (9,6), legend = True)
test_set.y.plot(legend = True)
predictions.plot(legend = True)

In [ ]:
#MAPE
def MAPE(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
MAPE(test_set.y, predictions)

#Future



In [ ]:
# Create a dictionary with data
data = {
            'High': [4.123123],
    'Low': [4.123123],
    'y': [210.935516],
    'Volume': [4.123123],
        'trend': [8.12313],
        'weekly': [4.123123],
            'multiplicative_terms': [4.444],
                

                
                }

# Create a new DataFrame object
df = pd.DataFrame(data)


In [ ]:
Future = xgb.DMatrix(df, label = df.y)
f_predictions = pd.Series(model.predict(Future), name = "XGBoost")
f_predictions